In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [5]:
movies = pd.read_csv("../datasets/reseñas_limpio.csv") #importo mi csv limpio


PARA ENTRENAR EL MODELO, PRIMERO NECESITAMOS ETIQUETAR CON 1 Y 0 LA COLUMNA DE SENTIMENT.

In [3]:
y = movies['sentiment'] #NUESTRA VARIABLE Y CON 0 Y 1 PARA LAS ETIQUETAS DEL SENTIMIENTO

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y))) #CON ESTO LO QUE HAGO ES ESTABLECER EN 1 LOS POSITIVOS 
#Y EN 0 LOS NEGATIVOS.

X = movies["review"] #LA VARIABLE X SERAN LAS LISTAS DE PALABRAS DE RESEÑAS

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#ESTABLEZCO UNA PROPORCION 0.20 PARA EL CONJUNTO DE PRUEBA Y 0.80 PARA EL DE ENTRENAMIENTO.

In [5]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)#compruebo que todas las dimensiones están bien.

(40000,) (10000,) (40000,) (10000,)


In [6]:
from keras.preprocessing.text import Tokenizer

In [7]:
token = Tokenizer(num_words=5000)  #establezco el número máximo de palabras que se pueden conservar, según la frecuencia de palabras.
token.fit_on_texts(X_train) #actualizo el vocabulario en el x train

X_train = token.texts_to_sequences(X_train) #transformamos nuestros tokens en numeros para x train y para x test
X_test = token.texts_to_sequences(X_test)


In [8]:
#debemos establecer ahora un tamaño igual para todas las listas, ya que hay reseñas mas largas o menos largas asique vamos
#a establecer una longitud de 100.

In [9]:

vocab_size = len(token.word_index) + 1 #Todas las palabras y sus índices se almacenarán en un diccionario al que podre acceder 



X_train = pad_sequences(X_train, padding='post', maxlen=100) #dejo un maximo de 100 palabras por lista.
X_test = pad_sequences(X_test, padding='post', maxlen=100)
vocab_size #nuestro corpus tiene 91098 palabras únicas.


91098

In [10]:
X_train.shape #ahora tenemos 40000 listas con 100 palabras

(40000, 100)

In [11]:
X_test.shape #igual que en el test.

(10000, 100)

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

**Modelo simple multicapa de Perceptron**

In [13]:
model = Sequential()
model.add(Embedding(5000, 32, input_length=100))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 32)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 250)               800250    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 251       
Total params: 960,501
Trainable params: 960,501
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=32,
verbose=1)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Precisión: %.2f%%" % (scores[1]*100))

1250/1250 [==============================] - 20s 16ms/step - loss: 0.3561 - accuracy: 0.8382 - val_loss: 0.3007 - val_accuracy: 0.8735
Precisión: 87.35%


In [15]:
model.evaluate(X_train, y_train, verbose=0)[1] 

0.9405999779701233

In [16]:
model.evaluate(X_test,y_test,verbose=0)[1]

0.8734999895095825

In [30]:
#OBSERVAMOS QUE EL MODELO GENERALIZA BASTANTE BIEN YA QUE HAY POCA DIFERENCIA ENTRE COMO APRENDIÓ CON EL ENTRENAMIENTO Y 
#CON LA PARTE DE PRUEBA.

AHORA DEBEMOS REVERTIR LOS NUMEROS A LA LISTA DE TEXTOS ORIGINAL PARA SABER SI ACERTÓ CON LAS PREDICCIONES, YA QUE SI NO 
HACEMOS ESTA CONVERSIÓN NO PODEMOS SABER POR LOS NÚMEROS QUE CONTEXTO TIENE ESA RESEÑA.

In [18]:
train_reviews = token.sequences_to_texts(X_train) 

In [19]:
test_reviews = token.sequences_to_texts(X_test) #paso 

In [58]:
pred_test = model.predict(X_test) #HAGO PREDICCIONES EN EL CONJUNTO DE PRUEBA 
pred_test[9] #COJEMOS EL NUMERO 9 Y VEMOS QUE POR LA PUNTUACIÓN, AL ESTAR MÁS CERCA DEL 0, CASI PEGADO, NOS ACLARA QUE ES
#UNA RESEÑA MUY NEGATIVA. 

array([0.01607266], dtype=float32)

In [21]:
test_reviews[9]  #podemos observar que por el contexto de las palabras, la reseña es negativa, y además bastante, como
#predijo bien nuestro modelo puntuándola bajisima.

"'offered' 'hope' 'decided' 'give' 'minutes' 'introduction' 'film' 'director' 'added' 'bit' 'solid' 'acting' 'performance' 'raised' 'presence' 'beyond' 'welcome' 'eye' 'candy' 'ultimately' 'obvious' 'low' 'budget' 'film' 'poorly' 'shot' 'scenes' 'pace' 'slapstick' 'certain' 'moments' 'favourite' 'movies' 'time' 'low' 'budget' 'one' 'also' 'deals' 'guys' 'dream' 'luck' 'however' 'money' 'actors' 'save' 'terrible' 'movie' 'could' 'cult' 'movie' 'laughed' 'loud' 'scenes' 'involving' 'joe' 'character' 'particular' 'scenes' 'terribly' 'clich' 'still' 'funny' 'rich' 'characters' 'house' 'story' 'towards' 'final' 'moments' 'see' 'great' 'stage' 'play' 'film' 'makers' 'best' 'translate' 'celluloid' 'simply' 'work' 'laughed' 'loud' 'scenes' 'one' 'liners' 'think' 'first' 'minutes' 'expectations' 'degree' 'would' 'laughed' 'anything' 'unless' 'stuck' 'coffee' 'pick' 'see' 'bargain'"

In [22]:
#probemos con alguna positiva.

In [57]:
pred_test[25] #esta la puntúan positiva

array([0.76632994], dtype=float32)

In [56]:
test_reviews[25] #tampoco se equivocó ya que la ponen bien por el contexto.

"'went' 'see' 'hamlet' 'jobs' 'figured' 'hours' 'would' 'great' 'fan' 'branagh' 'dead' 'henry' 'completely' 'direction' 'acting' 'cinematography' 'film' 'captured' 'like' 'reviews' 'hours' 'passes' 'branagh' 'play' 'hamlet' 'hamlet' 'born' 'watch' 'film' 'constantly' 'trying' 'find' 'looked' 'noticed' 'able' 'move' 'camera' 'hall' 'mystery' 'movie' 'shot' 'shame' 'columbia' 'released' 'widescreen' 'version' 'vhs' 'dvd' 'player' 'take' 'titanic' 'day' 'columbia' 'listening' 'put' 'film' 'way' 'watched' 'know' 'happened' 'oscars' 'best' 'picture' 'best' 'actor' 'best' 'direction' 'best' 'cinematography' 'films' 'watching' 'felt' 'sorry' 'branagh' 'oscars' 'tribute' 'shakespeare' 'screen' 'giving' 'tribute' 'branagh' 'bringing' 'one' 'greatest' 'films' 'time'"

In [34]:
#VAMOS A GUARDAR NUESTRO MODELO

In [54]:

model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
#serializan los pesos (weights) para HDF5
model.save_weights("model.h5")
print("Modelo guardado en el PC")

Modelo guardado en el PC
